# Limitation of basic orbit propagation for investigating low flybys

The official trajectories in the Horizons repository are derived from actual two-way Doppler tracking data and include the effects of Earth's precession, variations in Earth's gravity, solar and lunar gravitational influences, solar wind effects, relativistic effects, etc. The time and effort needed to account for all such effects well enough to cross-check JPL and ESA OD would be hard ot justify merely for investigating the anomaly.

Code below tests orbit propagation across the 2 hour 39 minute gap in the two-way Doppler tracking of NEAR's flyby. The difference of 3.5 m/s in range rate far exceeds NEAR's anomalous $\Delta v$ of 13.46 mm/s. Only those properties of the anomaly can be tested that concern at most the single attractor dynamics and signal propagation.

In [1]:
from astropy import units as u
from astropy.time import Time
from poliastro.util import norm
from poliastro.frames import Planes
from poliastro.ephem import Ephem
from poliastro.bodies import Earth

from poliastro.twobody.orbit import Orbit
from poliastro.twobody.sampling import EpochsArray

from astropy.coordinates import solar_system_ephemeris

import numpy as np
import sys
sys.path.append('../')

from sim.stations import dss25, dss34
from sim.tracking import Tracking
from sim.util import describe_orbit, describe_state, describe_trajectory, compare_orbits

solar_system_ephemeris.set("de440")
goldstone_end = Tracking.NEAR_GOLDSTONE_END.value
canberra_start = Tracking.NEAR_CANBERRA_START.value

In [2]:
near_goldstone_ephem = Ephem.from_horizons("NEAR", goldstone_end, attractor=Earth, plane=Planes.EARTH_EQUATOR)
near_goldstone_end_rv = near_goldstone_ephem.rv(goldstone_end)
describe_state(near_goldstone_end_rv, dss25, goldstone_end)

near_goldstone_orbit = Orbit.from_vectors(Earth, near_goldstone_end_rv[0], near_goldstone_end_rv[1], goldstone_end)
describe_orbit(near_goldstone_orbit)


:AT: 1998-01-23 06:14:55.600
Geocentric distance: 37471.700215992445 km speed: 8.258344936971223 km / s
From Goldstone-25 : range  31576.174442159892 km , range rate  -7.684513660227914 km / s

::ORBIT::
Plane: Planes.EARTH_EQUATOR
Inclination: 107.97266698121356 deg
Eccentricity: 1.813357091534396
Semilatus rectum: 19437.250829758486 km
Semimajor axix: -8494.322039490264 km
Periapse radius: 6908.917068596319 km , altitude: 530.7804685963192 km


In [3]:
near_canberra_ephem = Ephem.from_horizons("NEAR", canberra_start, attractor=Earth, plane=Planes.EARTH_EQUATOR)
near_canberra_start_rv = near_canberra_ephem.rv(canberra_start)
describe_state(near_canberra_start_rv, dss34, canberra_start)

near_canberra_orbit = Orbit.from_vectors(Earth, near_canberra_start_rv[0], near_canberra_start_rv[1], canberra_start)
describe_orbit(near_canberra_orbit)


:AT: 1998-01-23 09:53:55.600
Geocentric distance: 74714.76607137504 km speed: 7.589138197919106 km / s
From Canberra-34 : range  73147.47485388452 km , range rate  7.32170013250344 km / s

::ORBIT::
Plane: Planes.EARTH_EQUATOR
Inclination: 107.97095856088998 deg
Eccentricity: 1.8134615874116717
Semilatus rectum: 19440.644557497064 km
Semimajor axix: -8494.398279001718 km
Periapse radius: 6909.86670814371 km , altitude: 531.7301081437099 km


In [4]:
compare_orbits(near_goldstone_orbit, near_canberra_orbit)


Δ Inclination: -0.0017084203235683691 deg
Δ Eccentricity: 0.0001044958772757898
Δ Semilatus rectum: 3.3937277385775815 km
Δ Semimajor axix: -0.07623951145433239 km
Δ Periapse radius: 0.9496395473906887 km
Δ Energy: -0.000210584568196948 km2 / s2


In [5]:
sampling_interval = 180
sample_offsets = np.arange(0, (canberra_start-goldstone_end)/(1*u.s), sampling_interval)
flyby_epochs = goldstone_end + (sample_offsets << u.s)
print("Start and end epochs:", goldstone_end, flyby_epochs[-1], "[", flyby_epochs.size, "]")

Start and end epochs: 1998-01-23 06:14:55.600 1998-01-23 09:53:55.600 [ 74 ]


In [6]:
near_goldstone_extended_ephem = near_goldstone_orbit.to_ephem(EpochsArray(flyby_epochs))
describe_trajectory(near_goldstone_extended_ephem, dss34)


::TRAJECTORY::
Start and end ranges: 37309.07130107391 km 73145.82143191747 km
Start and end radial speeds: -7.97119272554438 km / s 7.3216176182785535 km / s
Closest: 11326.436162795784 km speed 12.740147538216107 km / s  at  1998-01-23 07:23:55.600
Deflection: 64.92614895100142 deg


In [7]:
goldstone_predicted_canberra_rv = near_goldstone_extended_ephem.rv(canberra_start)
describe_state(goldstone_predicted_canberra_rv, dss34, canberra_start)


:AT: 1998-01-23 09:53:55.600
Geocentric distance: 74715.16140743632 km speed: 7.5891622264313146 km / s
From Canberra-34 : range  73145.82143191747 km , range rate  7.3216176182785535 km / s


In [8]:
print(norm(goldstone_predicted_canberra_rv[0]-near_canberra_start_rv[0]) << u.km, norm(goldstone_predicted_canberra_rv[1]-near_canberra_start_rv[1]) << (u.m/u.s))

35.71831820614084 km 3.4529739544187095 m / s
